In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

In [ ]:
df=pd.read_csv('/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [ ]:
df.head(5)

In [ ]:
df.shape

In [ ]:
df.columns.values

In [ ]:
df.dtypes

#TotalCharges have string as dtype which is wrong

In [ ]:
df.describe()

In [ ]:
churn_count=df['Churn'].value_counts()
churn_count

In [ ]:
churn_count.plot(kind='bar',figsize=(6,4))
plt.xlabel('Churn')
plt.ylabel('Count')
plt.title('Count of customers')

In [ ]:
df.info()

In [ ]:
new_df=df.copy()

In [ ]:
new_df.TotalCharges=pd.to_numeric(new_df.TotalCharges,errors='coerce')
new_df.info()

Now Total carges is in float but has some missing values so we will fill it with mean value

In [ ]:
new_df.TotalCharges.fillna(new_df.TotalCharges.mean())

In [ ]:
new_df.tenure.max()

In [ ]:
lables=["{0}-{1}".format(i,i+11) for i in range(1,72,12)]
new_df['Tenure_group']=pd.cut(new_df.tenure,range(1,80,12),right=False,labels=lables)

In [ ]:
new_df.Tenure_group.value_counts()

In [ ]:
new_df.drop(columns=['tenure','customerID'],axis=1,inplace=True)

In [ ]:
new_df.head()

In [ ]:
for i, predictor in enumerate(new_df.drop(columns=['Churn', 'TotalCharges', 'MonthlyCharges'])):
    plt.figure(i)
    ax = sns.countplot(data=new_df, x=predictor, hue='Churn')
    
    # Annotate the bars with their counts
    for p in ax.patches:
        count = int(p.get_height())  # Get the height of the bar
        if count > 0:  # Only annotate if count is greater than 0
            ax.text(p.get_x() + p.get_width() / 2.,  # X-coordinate
                    p.get_height() + 0.5,           # Y-coordinate
                    f'{count}',                    # Text to display
                    ha='center',                   # Horizontal alignment
                    va='center',                   # Vertical alignment
                    fontsize=10)                   # Font size
    
    plt.title(f'Countplot of {predictor} by Churn')
    plt.show()


### Univariate Analysis

- **Gender**: The male-to-female ratio is the **same** for churned and non-churned customers.  
- **Senior Citizen**: **Senior Citizens** are significantly more likely to churn.(71%)  
- **Partner**: Customers with **no partner** are more likely to churn.(50%)
- **Dependents**: Customers with **no dependents** are more likely to churn.(45%)
- **Phone Services**: The rato is **same**
- **Multiple Lines**: The rato is **same**
- **Internet Service**: Customer with **fiber optic** is more likely to churn(72%)
- **Online Security**:Customer with **No Security** is more likely to churn(71%)
- **Online Backup**:Customer with **No Backup** is more likely to churn(66%)
- **Device Protection**:Customer with **No Protection** is more likely to churn(64%)
- **Tech Support**:Customer with **No Support** is more likely to churn(71%)
- **Streaming Tv**: The rato is **same**
- **Streaming Movies**: The rato is **same**
- **Contract**:Customer with **Month-to-Month contract** is more likely to churn(74%)
- **Paperless Billing**:Customer with **Paperless Billing** is more likely to churn(50%)
- **Payment Method**:Customer with **Electronic Check** is more likely to churn(83%)
- **Tenure Group**:Customer with Tenure Group **1-12** is more likely to churn(91%)

In [ ]:
new_df_dummies=pd.get_dummies(new_df)
new_df_dummies.head()

In [ ]:
sns.lmplot(data=new_df_dummies,x='MonthlyCharges',y='TotalCharges',fit_reg=False)

In [ ]:
new_df_dummies['MonthlyCharges'].corr(new_df_dummies['TotalCharges'])

**TotalCharges increases as the MonthlyCharges increases**

In [ ]:
Mth=sns.kdeplot(new_df_dummies.MonthlyCharges[(new_df_dummies['Churn_No'])],
               color='Red',shade=True)
Mth=sns.kdeplot(new_df_dummies.MonthlyCharges[(new_df_dummies['Churn_Yes'])],
               ax=Mth,color='Blue',shade=True)
Mth.legend(['No Churn','Churn'],loc='upper right')
Mth.set_xlabel('Density')
Mth.set_ylabel('Monthly Charges')
Mth.set_title('Monthly Charges by churn')

**Churn is high when monthly charges are high**

In [ ]:
tot=sns.kdeplot(new_df_dummies.TotalCharges[(new_df_dummies['Churn_No'])],
               color='Red',shade=True)
tot=sns.kdeplot(new_df_dummies.TotalCharges[(new_df_dummies['Churn_Yes'])],
               ax=tot,color='Blue',shade=True)
tot.legend(['No Churn','Churn'],loc='upper right')
tot.set_xlabel('Density')
tot.set_ylabel('Total Charges')
tot.set_title('Total Charges by churn')